# Chapter 1: Introduction to LLM Evaluation

## Setup

In [1]:
import weave
from set_env import set_env
import nest_asyncio
import json
import asyncio

In [ ]:
set_env("GOOGLE_API_KEY")
set_env("WANDB_API_KEY")
print("Env set")

In [3]:
try:
    import IPython
    in_jupyter = True
except ImportError:
    in_jupyter = False
if in_jupyter:
    nest_asyncio.apply()

In [6]:
from utils.config import WEAVE_PROJECT, ENTITY, MODEL, MODEL_CLIENT
from utils.prompts import medical_task, medical_system_prompt 
from utils.render import display_prompt, print_dialogue_data
from utils.llm_client import LLMClient
from utils.prompts import medical_privacy_judge_prompt, MedicalPrivacyJudgement, medical_task_score_prompt, MedicalTaskScoreJudgement, medical_task_score_system_prompt, medical_privacy_system_prompt
from utils.evals import get_evaluation_predictions, calculate_kappa_scores, calculate_weighted_alignment

## Understanding Medical Data Extraction Evaluation

### The Task: What Are We Trying to Do?

#### Raw Data Format
Medical conversations are messy and unstructured. Looking at our example data:

1. **Dialogue Format**:
- Back-and-forth conversation between doctor and patient
- Contains personal details, small talk, and medical information mixed together
- Informal language ("hey", "mm-hmm", "yeah")
- Important details scattered throughout

2. **Medical Notes**:
- More structured but still in prose
- Contains standardized sections (CHIEF COMPLAINT, HISTORY, etc.)
- Includes sensitive information (names, ages)
- Medical terminology and abbreviations

#### Extraction Goals
The LLM needs to:
1. Find relevant information
2. Ignore irrelevant details
3. Standardize the format
4. Protect patient privacy
5. Maintain medical accuracy

In [ ]:
weave_client = weave.init(f"{ENTITY}/{WEAVE_PROJECT}")

In [ ]:
display_prompt(medical_system_prompt)
display_prompt(medical_task)

<div align="center">
    <img src="./media/medical_chatbot.png" width="250"/>
</div>

In [ ]:
annotated_medical_data = weave.ref(f"weave:///{ENTITY}/{WEAVE_PROJECT}/object/medical_data_annotations:latest").get()
# annotated_medical_data = weave.ref("weave:///a-sh0ts/eval_course_ch1_dev/object/medical_data_annotations:At9gri9UasftpPe5VNzT3EuIXQWAo5MYX8aMf2cuE8A").get()

In [ ]:
print_dialogue_data(annotated_medical_data, indexes_to_show=[0])


### In fact, let's just generate an example now:

In [ ]:
llm = LLMClient(model_name=MODEL, client_type=MODEL_CLIENT)
llm.predict(user_prompt=medical_task.format(transcript=annotated_medical_data[0][0]["input"]), system_prompt=medical_system_prompt)



## Data Collection and Curation for Evaluation

### Best practices for medical extraction evaluation:
1. Collect real medical transcripts and LLM outputs
2. Include diverse medical conditions and conversation styles
3. Balance routine vs complex medical cases
4. Remove duplicate records
5. Validate with medical experts

These become our evaluation dataset

In [ ]:
print_dialogue_data(annotated_medical_data, indexes_to_show=[1])


## Why and How to Evaluate LLMs

### Core Principles of LLM Evaluation
Unlike traditional software testing, LLM evaluation requires special consideration:

1. **Non-Deterministic Outputs**
   - Models can give different valid answers
   - Responses vary between runs
   - Multiple correct solutions possible

2. **Quality is Multi-Dimensional**
   - Correctness isn't binary
   - Context matters heavily
   - Different stakeholders have different priorities

3. **Scale vs Accuracy Trade-offs**
   - Manual review is accurate but expensive
   - Automated checks are scalable but limited
   - Hybrid approaches often work best

### Practical Evaluation Recipe 🧑‍🍳

1. **Define Success Criteria**
   - List must-have requirements
   - Set acceptable thresholds
   - Identify critical failures

2. **Build Evaluation Suite**
   - Automated checks for clear rules
   - Expert review for nuanced cases
   - Version control evaluation code

3. **Create Scoring System**
   - Weight different factors
   - Establish baselines
   - Plan for aggregation

### Applying to Medical Data Extraction 🏥

For our medical extraction task, this means:
- **Success Criteria**: Required fields, privacy compliance, word limits
- **Evaluation Suite**: Automated checks + medical expert review
- **Scoring**: Weighted combination of format, accuracy, and safety metrics

Let's see how to implement this...

![](./media/traditional_llm_eval.png)

## First: Annotation: Building Quality Training Data

### Why Annotate?
To evaluate LLMs effectively, we need expert-labeled data that:
1. Defines what "good" looks like
2. Shows us what to test for
3. Helps align our automated tests with human judgment

### The Process
Experts review outputs and provide structured feedback. This creates a foundation for:
- Building automated evaluation tests
- Measuring how well those tests match expert judgment
- Refining our evaluation methods until they align with expert standards

Think of annotations as our compass - they help ensure our later automated evaluation methods point in the same direction as human experts while assessing the quality of our LLM's outputs.

<div align="center">
    <img src="./media/annotation_ui.png" width="450"/>
</div>

In [ ]:
print_dialogue_data(annotated_medical_data, indexes_to_show=[2, 3, 4])

## Evaluation: Measuring Performance

### Understanding LLM Evaluation
Unlike traditional software testing, LLM evaluation requires multiple approaches:

1. **Automated Checks**
   - Fast, programmatic tests
   - Clear pass/fail criteria
   - Example: format rules, required fields

2. **Model-Assisted Evaluation**
   - Using LLMs to evaluate outputs
   - Helpful for subjective criteria
   - Example: checking medical accuracy, privacy compliance

3. **Expert Review**
   - Human validation of complex cases
   - Ground truth for training evaluators
   - Example: annotated datasets

### Building Evaluation Systems

In this notebook, we'll implement this through:

1. **Basic Tests**
   ```python
   test_adheres_to_required_keys()
   test_adheres_to_word_limit()
   ```

2. **LLM Judges**
   ```python
   judge_adheres_to_privacy_guidelines()
   judge_overall_score()
   ```

3. **Key Questions**
   - How closely do automated evaluations match human judgment?
   - When do automated systems diverge from human experts?
   - What makes a good evaluation system?

These questions lead us to the concept of alignment - measuring how well our automated systems match human expectations and values. We'll explore practical ways to measure and improve this alignment after implementing our evaluation system.

![](./media/eval_task_flowchart.png)

### Using Domain Knowledge to Build Evaluation Tests

We'll create four key tests to evaluate our medical extraction outputs:

1. **Required Fields Check**
   - Verifies presence of essential medical fields
   - E.g., "Chief complaint", "Symptoms", "Follow-up instructions"

2. **Word Limit Check**
   - Ensures output stays within 150-word limit
   - Promotes concise, focused summaries

3. **Privacy Guidelines Check**
   - Uses LLM to detect any PII leakage
   - Critical for medical data compliance

4. **Overall Quality Score**
   - LLM-based assessment of extraction quality
   - Considers accuracy, completeness, and format

These tests will be validated against our expert-annotated dataset to ensure they align with human judgment. This alignment process helps us understand how well our automated evaluation matches medical expert standards.

Let's implement each test:

In [15]:
test_output = annotated_medical_data[0][1]["output"]

In [17]:
@weave.op()
def test_adheres_to_required_keys(model_output: str):
    # Required medical keys
    required_keys = [
        "Chief complaint",
        "History of present illness",
        "Physical examination",
        "Symptoms",
        "New medications with dosages",
        "Follow-up instructions"
    ]
    
    # Convert to lowercase for case-insensitive matching
    model_output_lower = model_output.lower()
    
    # Check if all required keys are present
    for key in required_keys:
        if key.lower() not in model_output_lower:
            return int(False)
            
    return int(True)

In [ ]:
test_adheres_to_required_keys(test_output)

In [19]:
@weave.op()
def test_adheres_to_word_limit(model_output: str):
    return int(len(model_output.split()) <= 150)

In [ ]:
test_adheres_to_word_limit(test_output)

In [ ]:
display_prompt(medical_privacy_system_prompt)
display_prompt(medical_privacy_judge_prompt)

In [22]:
@weave.op()
def judge_adheres_to_privacy_guidelines(model_output: str):
    llm = LLMClient(model_name=MODEL, client_type=MODEL_CLIENT)
    response = llm.predict(user_prompt=medical_privacy_judge_prompt.format(text=model_output), system_prompt=medical_privacy_system_prompt, schema=MedicalPrivacyJudgement)
    try:
        result = json.loads(response.text.strip("\n"))
        return int(not result[0]["contains_pii"])
    except:
        return int(True) #TODO: Add json parsing as failure reason

In [ ]:
judge_adheres_to_privacy_guidelines(test_output)

In [ ]:
display_prompt(medical_task_score_system_prompt)
display_prompt(medical_task_score_prompt)

In [25]:
@weave.op()
def judge_overall_score(model_output: str):
    llm = LLMClient(model_name=MODEL, client_type=MODEL_CLIENT)
    response = llm.predict(user_prompt=medical_task_score_prompt.format(text=model_output), system_prompt=medical_task_score_system_prompt, schema=MedicalTaskScoreJudgement)
    try:
        result = json.loads(response.text.strip("\n"))
        return result[0]["score"]
    except:
        return 0 #TODO: Add json parsing as failure reason


In [ ]:
judge_overall_score(test_output)

### We already have a dataset of annotated medical data. We can use our tests to evaluate the outputs of our LLM.

In [27]:
@weave.op()
def annotated_data_passthrough(input, output):
    return output

In [ ]:
annotated_medical_data[0][2]

In [29]:
evaluation_data = [
    {"input": annotated_row[0]["input"], "output": annotated_row[1]["output"], "scores": {"human_required_keys": annotated_row[3]["presence_of_keys"], "human_word_limit": annotated_row[3]["word_count"], "human_absence_of_PII": annotated_row[3]["absence_of_PII"], "human_overall_score": annotated_row[2]}}
    for annotated_row in annotated_medical_data
][0:5]

In [ ]:
# Create evaluation
evaluation = weave.Evaluation(
    dataset=evaluation_data,
    scorers=[test_adheres_to_required_keys, test_adheres_to_word_limit, judge_adheres_to_privacy_guidelines, judge_overall_score]
)

# Run evaluation
evals = asyncio.run(evaluation.evaluate(annotated_data_passthrough))

### But do our test outputs adhere to the annotation expectations?

We need to measure how well our automated evaluations match human judgment. We'll:

1. **Measure Alignment**
   - Compare automated test results with expert annotations using kappa scores
   - Weight different aspects based on their importance (privacy, completeness, etc.)
   - Find where automated tests disagree with human experts

2. **Use These Results**
   - Chapter 2 will focus on improving the LLM judges that show poor alignment
   - We'll learn to refine prompts based on these alignment scores
   - Build better evaluation systems by focusing on the weakest areas first

These alignment measurements are crucial - they tell us which parts of our automated system need the most work, especially for critical aspects like privacy checks and medical accuracy.

In [35]:
eval_call_id = "0192e825-82fe-7c03-b03d-9eabe0c81ff5"

In [ ]:
df = get_evaluation_predictions(weave_client, eval_call_id)
df

#### Software Tests: Minimal Alignment and hard to optimize

In [ ]:
kappa_scores = calculate_kappa_scores(df[["input", "required_keys", "word_limit"]])
for metric, score in kappa_scores.items():
    print(f"{metric}: {score:.3f}")

#### LLM Judges: Higher Alignment and easier to optimize

In [ ]:
kappa_scores = calculate_kappa_scores(df[["input", "privacy", "overall"]])
for metric, score in kappa_scores.items():
    print(f"{metric}: {score:.3f}")

## Resources

- [Hamel's LLM Judge](https://hamel.dev/blog/posts/llm-judge/)
- [Hamel's LLM Evaluation](https://hamel.dev/blog/posts/evals/)
- [Clef's LLM Evaluation](https://huggingface.co/blog/clefourrier/llm-evaluation)
- [Eugene Yan's LLM Evaluators](https://eugeneyan.com/writing/llm-evaluators/)
- [Shreya's AI Engineering Flywheel](https://www.sh-reya.com/blog/ai-engineering-flywheel/)
- [Who Validates the Validators?](https://arxiv.org/abs/2404.12272)